Importing libraries for further usage

In [1]:
# removing warnings
import warnings
warnings.filterwarnings('ignore')

# computational needs
import networkx as nx
import pandas as pd

# scraping for data
import urllib.request
from bs4 import BeautifulSoup 

# plotting
from matplotlib import pyplot

# local files
import tools

# others
from unidecode import unidecode
import random, json
import numpy as np, numpy.random

useful methods for later

In [2]:
# CONSTANTS
MAX_JOBS_REFUGEES = 0.7

In [3]:
def get_job_potential( each):
    """Given a city, give the number of jobs found there."""
    
    # get the url for each city
    URL = "https://tr.indeed.com/jobs?l="+unidecode(each)    
    
    # extract the html, the values from the search
    soup = BeautifulSoup(urllib.request.urlopen(URL).read(), 'html.parser')
    values = soup.find("div", id="JOB_TYPE_rbo")
    
    # find total jobs found in this city
    if values == None: # if no job was found, skip
        return 0
    count = 0
    for li in values.find_all("li"):
        found_jobs = li.find("span", attrs={'class':"rbCount"}).text
        count += int( found_jobs[1:-1])
    return count

def update_info():
    """information about the job market in a all cities"""

    info = {}
    for n in G.nodes:
        # each refugee working knows 10 job openings
        jobs_info = 10 * G.nodes[n]['jobs_to_refugees']/G.nodes[n]['jobs']
        if jobs_info < 1 and jobs_info > 0.5:
            info[n] = jobs_info
        elif jobs_info <=0.5:
            info[n] = 0.5
        else:
            info[n] = 1.0
    return info

## Read City and Road data

In [4]:
# get city data from the two sources
cities  = pd.read_csv('tr.csv')
cities2 = pd.read_csv('city_data.csv',"\t")
cities.fillna(0, inplace=True)
# fix the Istanbul instance for later consistency
cities['city'][0] = 'İstanbul'

# merge the two data
city_data = pd.merge( cities, cities2, 
                     how='outer', left_on='city', right_on='Name')
# filter the columns you want
cities = city_data[['city','admin','Abbr.','lat','lng','Area(km²)',
                    'population', 'Population Estimate (E)2018-12-31']]
# rename the column
cities = cities.fillna('0').rename(columns={'Population Estimate (E)2018-12-31':'population2'}).iloc[:-2]
cities['population2'] = [ int(n.replace(",","")) for n in list(cities['population2'])]
cities['area'] = [ int(n.replace(",","")) for n in list(cities['Area(km²)'])]
cities['population'] = cities['population'].where( cities['population'] > 0, cities['population2'])

In [5]:
# get main roads in Turkey
main_roads = tools.get_cities_in_main_roads()

# cities to take in consideration
main_cities = list(cities[ cities['city'] == cities['admin']]['city'])
city_in_main_road = list(set( sum(main_roads, [])))

### Put these cities in the map

In [ ]:
# Put everything in a graph
G = nx.Graph()

all_cities = set( main_cities + city_in_main_road)
# create each city
for city in all_cities:
    
    # get number of jobs for this city
    job_potential = get_job_potential( city)
    
    # find position of city in map
    city_properties = cities[cities['city'] == city].iloc[0]
    
    # add the city to the graph
    G.add_node(city_properties['city'], 
               pos=( city_properties['lat'], city_properties['lng']), 
               size= int(city_properties['population']),
               area= int(city_properties['area']) if int(city_properties['area']) != 0 else 20000, 
               refugees=0, jobs_to_refugees=0,
               jobs=job_potential)

### Connect the cities

In [ ]:
# put the edges connecting the cities

# if found city in road, but not in data, fix manually
for road in main_roads:
    
    # if any of the paths is not in the list, then mention it, fix it manually
    for each in road:
        if each not in list(cities['city']):
            print("City not found in list: ",each) # TODO: deal automatically
            
    # remind the cities that are present
    for i in range(len(road)-1):
        # add a connection between each two cities
        G.add_edge( road[i], road[i+1], weight=10)
        
# connect the cities not in the list with the closest city in the list
not_found = set( main_cities)-set( city_in_main_road)

for each in not_found:
    city = cities[ cities['city'] == each].iloc[0]
    
    # order cities present in list by distance to this city
    found = cities[cities['city'].isin( all_cities)]
    found['distance'] = ((found['lat']-city['lat'])**2 + (found['lng']-city['lng'])**2)
    closest_city = found.sort_values('distance').iloc[1]['city'] # get the closest
    
    # add road connecting these cities
    G.add_edge( city['city'], closest_city, weight=10)

### Get the zone where the refugees enter, connect those cities

In [ ]:
# define the zone where the refugees enter
# entrance dates for the refugees
entrance_freq = pd.read_json("syrian_refugees.json")

# suggest area from Adana to Van
max_lat = cities[cities['city'] ==  'Van']['lat'].iloc[0]
max_lng = cities[cities['city'] ==  'Van']['lng'].iloc[0]
min_lng = cities[cities['city'] =='Adana']['lng'].iloc[0]

# filter
syrian_zone = list(cities[ ( cities[ 'lng'] <= max_lng) & 
                           ( cities[ 'lng'] >= min_lng) & 
                           ( cities[ 'lat'] <= max_lat) & 
                           ( cities['city'] == cities['admin'])]['city'])

# connect all cities close to Syria with the closest cities in this area
for each in syrian_zone:
    
    city = cities[ cities['city'] == each].iloc[0]
    
    # order cities present in list by distance to this city
    found = cities[cities['city'].isin( [n for n in syrian_zone if n != each])]
    found['distance'] = ((found['lat']-city['lat'])**2 + (found['lng']-city['lng'])**2)
    closest_cities = found.sort_values('distance') 
    
    # add road connecting these cities
    # keep the weight lower to suggest less people can move in between
    G.add_edge( city['city'], closest_cities.iloc[0]['city'], weight=5)
    G.add_edge( city['city'], closest_cities.iloc[1]['city'], weight=5)
    G.add_edge( city['city'], closest_cities.iloc[2]['city'], weight=5)

In [ ]:
# Picture the map
pos = {city:(long, lat) for (city, (lat,long)) in nx.get_node_attributes(G, 'pos').items()}
nx.draw(G, pos, with_labels=False, node_size=10, node_color='g', edge_color='b')

### Start adding refugees depending on time

In [ ]:
# working potential
# males that are able to work are in the age range 18-59, 50% of them need a job
# females that are able to work are in the age range 18-59, 10 % of them can deal with a job
# what is the percentage of people able to work?
with open( "distribution.json") as f:
    d = json.load(f)
work_percentage = (int(d['data'][0]['male_1859'])*0.5 
                   + int(d['data'][0]['female_1859'])*0.1)/int(d['data'][0]['individuals'])
# work_percentage = 17%

In [ ]:
# distribute the refugees in the cities randomly
distribute_refugees = np.random.dirichlet(np.ones(len(syrian_zone)),size=1)[0] *\
                       int( entrance_freq['individuals'].iloc[0] * work_percentage)
refugees = [int(n) for n in distribute_refugees]

### Getting a job
It is easier to get a job, once other refugees work in the same city (Job information). <br>
If the city is small, the chances decrease. <br>
If the number of refugees in the city is really high in proportion to the city's population, then your chances decrease. 

In [ ]:
for i in range(len(syrian_zone)):
    n = syrian_zone[i]
    
    # put these people in the city
    G.nodes[ n]['refugees'] += refugees[i]
    
    # get the jobs that are available for the refugees once in city
    job_vacancy = int(MAX_JOBS_REFUGEES*G.nodes[n]['jobs']) - G.nodes[n]['jobs_to_refugees'] 
    G.nodes[n]['jobs_to_refugees'] += job_vacancy

In [ ]:
# for each round, for each city 
# give to the people not having a job the option to change city

# refugees without jobs in Adana
unemployed_refugees = G.nodes['Adana']['refugees'] - G.nodes['Adana']['jobs_to_refugees']
unemployed_refugees

In [ ]:
for city in all_cities:
    # get the neighbors of this city
    neighbors = [n for n in G.neighbors(city)] + [city]
    info = update_info() # get the job options there

    # decision: population, refugees, number of refugees employed, number of job positions
    # refugee/area, people/area, info about jobs
    decision = {}
    for each in neighbors:
        presence = G.nodes[each]['refugees']/G.nodes[each]['area']
        density = G.nodes[each]['size']/G.nodes[each]['area']
        jobs = info[each] * G.nodes[each]['jobs']
        decision[each] = presence * jobs/density

    # get the distribution of refugees moving out of this city
    move_refugees = [ int(decision[n]/sum( decision.values()) * unemployed_refugees) for n in decision]

    # move
    for i in range(len(neighbors)):
        G.nodes[n]['refugees'] += move_refugees[i]

    # remove the concentration
    G.nodes[city]['refugees'] -= sum(move_refugees)